In [1]:
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import gc

gc.enable()
sns.set(font_scale=1.5)
pd.set_option('display.max_columns', 130)
pd.set_option('display.max_rows', 130)

In [2]:
def df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [3]:
# Carregando nomes e populações de municípios, estados e regiões (IBGE - 2019)
popregiao = pd.read_csv('data/dim_municipio/pop_regioes.csv')
popestado = pd.read_csv('data/dim_municipio/pop_estados.csv')
popmunicipio = pd.read_csv('data/dim_municipio/pop_municipios.csv')

# Unindo tabela de estado à de regiões
popestado = pd.merge(popestado, popregiao, on='regiao')
popestado.rename(columns={'sigla': 'uf', 'populacao_x': 'pop_uf', 'populacao_y': 'pop_regiao'}, inplace=True)

# Adaptando tabela dos municípios
popmunicipio[['cod_uf', 'cod_munic']] = popmunicipio[['cod_uf', 'cod_munic']].astype('str')
popmunicipio['cod_munic'] = popmunicipio['cod_munic'].str.zfill(5)
popmunicipio['cod_ufmunicipio'] = popmunicipio['cod_uf'] + popmunicipio['cod_munic']
popmunicipio['6cod_municipio'] = popmunicipio.apply(lambda x: x['cod_ufmunicipio'][:-1], axis=1)

# Unindo tabela de estados com a de municípios
popmunicipio = pd.merge(popmunicipio, popestado, on='uf')
popmunicipio.drop(columns=['cod_uf', 'cod_munic', 'cod_ufmunicipio', 'estado'], axis=1, inplace=True)
popmunicipio.rename(columns={'populacao': 'pop_municipio'}, inplace=True)
popmunicipio = popmunicipio[['regiao', 'uf', 'municipio', 'pop_municipio', 'pop_uf', 'pop_regiao', '6cod_municipio']]

popmunicipio.head()

,regiao,uf,municipio,pop_municipio,pop_uf,pop_regiao,6cod_municipio
0,Norte,RO,Alta Floresta D'Oeste,22945,1777225,18430980,110001
1,Norte,RO,Ariquemes,107863,1777225,18430980,110002
2,Norte,RO,Cabixi,5312,1777225,18430980,110003
3,Norte,RO,Cacoal,85359,1777225,18430980,110004
4,Norte,RO,Cerejeiras,16323,1777225,18430980,110005


In [4]:
# Leitura de arquivos
medicos = pd.read_csv('data/dim_municipio/rh_medicos.csv')
enfermeiros = pd.read_csv('data/dim_municipio/rh_enfermeiros.csv')

tomografos_computadorizados = pd.read_csv('data/dim_municipio/rf_tomografos_computadorizados.csv')
ressonancia_magnetica = pd.read_csv('data/dim_municipio/rf_ressonancia_magnetica.csv')
raios_x = pd.read_csv('data/dim_municipio/rf_raios_x.csv')
mamografos = pd.read_csv('data/dim_municipio/rf_mamografos.csv')
leitos_internacao = pd.read_csv('data/dim_municipio/rf_leitos_internacao.csv')

nucleos_apoio_saude_familia = pd.read_csv('data/dim_municipio/nucleos_apoio_saude_familia.csv')
unidade_servico_apoio_diagnose_terapia = pd.read_csv('data/dim_municipio/estabelecimentos_unidade_servico_apoio_diagnose_terapia.csv')
ubs = pd.read_csv('data/dim_municipio/estabelecimentos_ubs.csv')
hospital_geral = pd.read_csv('data/dim_municipio/estabelecimentos_hospital_geral.csv')
hospital_especializado = pd.read_csv('data/dim_municipio/estabelecimentos_hospital_especializado.csv')
clinicas_ambulatorios_especializados = pd.read_csv('data/dim_municipio/estabelecimentos_clinicas_ambulatorios_especializados.csv')
equipes_saude_da_familia = pd.read_csv('data/dim_municipio/equipes_saude_da_familia.csv')
equipes_saude_da_familia.drop(columns=list(set(equipes_saude_da_familia.columns)-set(medicos.columns)), axis=1, inplace=True)

# Relação de dataframes
dims = [medicos, enfermeiros, tomografos_computadorizados, ressonancia_magnetica, raios_x, mamografos, leitos_internacao, nucleos_apoio_saude_familia, unidade_servico_apoio_diagnose_terapia, ubs, hospital_geral, hospital_especializado, clinicas_ambulatorios_especializados, equipes_saude_da_familia]

In [5]:
# Adicionando coluna para distinguir dataframes
for dataframe in dims:
    dataframe['tipo'] = df_name(dataframe)
    
# Checando número de colunas em todos os datasets
for dataframe in dims:
    print('Num of columns: {} - {}'.format(len(dataframe.columns), df_name(dataframe)))

Num of columns: 62 - medicos
Num of columns: 62 - enfermeiros
Num of columns: 62 - tomografos_computadorizados
Num of columns: 62 - ressonancia_magnetica
Num of columns: 62 - raios_x
Num of columns: 62 - mamografos
Num of columns: 62 - leitos_internacao
Num of columns: 62 - nucleos_apoio_saude_familia
Num of columns: 62 - unidade_servico_apoio_diagnose_terapia
Num of columns: 62 - ubs
Num of columns: 62 - hospital_geral
Num of columns: 62 - hospital_especializado
Num of columns: 62 - clinicas_ambulatorios_especializados
Num of columns: 62 - equipes_saude_da_familia


In [6]:
# Unindo todos os datasets
df = pd.concat(dims)

# Limpando col de municípios
df.rename(columns={'Munic�pio': 'municipio'}, inplace=True)
df['municipio'] = df.apply(lambda x: x['municipio'][:6], axis=1)

# Reordenando cols
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

# Substituindo vazios e caracteres especiais
df.replace({'-': 0}, inplace=True)
df.fillna(value=0, inplace=True)

# Consertando tipos de colunas numéricas
datacols = list(df.drop(columns=['tipo', 'municipio']).columns)
for colname in datacols:
    df[colname] = df[colname].astype(int)
    
# Enriquecendo com infos geográficas e demográficas
df = pd.merge(popmunicipio, df, how='right', left_on='6cod_municipio', right_on='municipio')
df.drop(columns=['municipio_y'], axis=1, inplace=True)
df.rename(columns={'municipio_x': 'municipio'}, inplace=True)

# Liberando 'base' da memória
del medicos, enfermeiros, tomografos_computadorizados, ressonancia_magnetica, raios_x, mamografos, leitos_internacao, nucleos_apoio_saude_familia, unidade_servico_apoio_diagnose_terapia, ubs, hospital_geral, hospital_especializado, clinicas_ambulatorios_especializados, equipes_saude_da_familia
gc.collect()

print('Tamanho original do dataset geral: {}'.format(len(df)))
df.head()

Tamanho original do dataset geral: 78361


,regiao,uf,municipio,pop_municipio,pop_uf,pop_regiao,6cod_municipio,tipo,2014/Jan,2014/Fev,2014/Mar,2014/Abr,2014/Mai,2014/Jun,2014/Jul,2014/Ago,2014/Set,2014/Out,2014/Nov,2014/Dez,2015/Jan,2015/Fev,2015/Mar,2015/Abr,2015/Mai,2015/Jun,2015/Jul,2015/Ago,2015/Set,2015/Out,2015/Nov,2015/Dez,2016/Jan,2016/Fev,2016/Mar,2016/Abr,2016/Mai,2016/Jun,2016/Jul,2016/Ago,2016/Set,2016/Out,2016/Nov,2016/Dez,2017/Jan,2017/Fev,2017/Mar,2017/Abr,2017/Mai,2017/Jun,2017/Jul,2017/Ago,2017/Set,2017/Out,2017/Nov,2017/Dez,2018/Jan,2018/Fev,2018/Mar,2018/Abr,2018/Mai,2018/Jun,2018/Jul,2018/Ago,2018/Set,2018/Out,2018/Nov,2018/Dez
0,Norte,RO,Alta Floresta D'Oeste,22945.0,1777225.0,18430980.0,110001,medicos,9,10,10,9,11,10,9,9,9,10,11,11,12,16,13,13,13,12,15,15,15,15,16,17,17,17,16,16,16,16,16,17,17,17,17,17,17,17,17,18,18,17,17,15,15,15,16,17,17,19,18,17,17,17,17,16,17,17,16,15
1,Norte,RO,Alta Floresta D'Oeste,22945.0,1777225.0,18430980.0,110001,enfermeiros,14,14,16,16,16,16,16,16,16,16,16,16,15,15,14,14,14,14,14,14,14,13,13,15,14,17,17,17,17,17,17,17,17,17,17,17,17,18,17,17,17,17,18,17,17,17,17,19,19,19,20,20,20,20,20,20,19,19,19,19
2,Norte,RO,Alta Floresta D'Oeste,22945.0,1777225.0,18430980.0,110001,tomografos_computadorizados,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Norte,RO,Alta Floresta D'Oeste,22945.0,1777225.0,18430980.0,110001,ressonancia_magnetica,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Norte,RO,Alta Floresta D'Oeste,22945.0,1777225.0,18430980.0,110001,raios_x,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [7]:
df.to_csv('infos_integradas_municipios.csv', index=False)